### Setup

In [1014]:
import nbformat
import extensions.my_ext

# import importlib
# importlib.reload(extensions.my_ext)

extensions.my_ext.load_ipython_extension(get_ipython())

def make(fname):
    source = open(fname, 'rb').read()
    nb = nbformat.reads(source.decode('utf8'), as_version=4)
    parents = {}
    cells = [extensions.my_ext.NotebookCell(cell, parents, index) for index, cell in enumerate(nb.cells)]
    with open('/data/sets/server/gchatstats/static/view.html', 'w') as f:
        f.write(make_html(cells))

def make_html(cells):
    header_html = ''
    body_html = ''
    jsx_scripts = ''
    scripts = ''
    run_scripts = ''
    global_error = ''
    for cell in cells:
        if cell.magic == '%%js' and '@out' in cell.labels:
            scripts += cell.get_output('application/javascript') + '\n'
        if cell.magic == '%%js' and '@out_run' in cell.labels:
            run_scripts += cell.get_output('application/javascript') + '\n'
        if cell.magic == '%%js' and '@out_error' in cell.labels:
            global_error += cell.source #('application/javascript') + '\n'
        elif cell.magic == '%%jsx' and '@out' in cell.labels:
            jsx_scripts += cell.get_output('application/javascript') + '\n'
        elif cell.magic == '%%css' and '@out' in cell.labels:
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%html' and '@header' in cell.labels:
            header_html += cell.source + '\n'
        elif cell.magic == '%%html' and '@body' in cell.labels:
            body_html += cell.source + '\n'
        elif cell.kind == 'code' and cell.source.startswith('%css '):
            header_html += cell.get_output('text/html') + '\n'
        elif cell.magic == '%%requirejs':
            require_config = cell.get_output('application/javascript') + '\n'
    res = '<!DOCTYPE html><html><head>\n'
    res += header_html + '\n'
    if hasattr(extensions.my_ext, 'requirejs'):
        res += '<script src=\"%s\"></script>\n' % extensions.my_ext.requirejs['lib_path']
    res += '<script>\n' + require_config + '\n</script>\n'
    res += '<script>\n' + global_error + '\n</script>\n'
    res += '<script>\n' + scripts + '\n</script>\n'
    res += '<script>\n' + jsx_scripts + '\n</script>\n'
    res += '</head><body>\n'
    res += body_html
    res += '<script>\n' + run_scripts + '\n</script>\n'
    res += '</body></html>\n'
    return res

In [1015]:
%%requirejs /static/requirejs@2.3.1/require.min.js
$           jquery  /static/jquery@2.1.1/jquery.min.js
React       react   /static/react@15.3.1/build/react.min.js
ReactDOM            /static/react@15.3.1/build/react-dom.min.js
R                   /static/ramda@0.22.1/ramda.min.js
vis                 /static/visjs@4.16.1/dist/vis.min.js

<IPython.core.display.Javascript object>

In [1016]:
%%js @out_error
require.config({
  waitSeconds : 60
})

<IPython.core.display.Javascript object>

In [1017]:
%%html @header
<meta charset="utf-8">
<title>Загрузка...</title>
<link href="https://fonts.googleapis.com/css?family=Roboto:400,300,500,600,700&subset=latin,cyrillic" rel="stylesheet" type="text/css">
<link rel="stylesheet" href="/static/fontawesome@4.6.3/css/font-awesome.min.css">

In [1018]:
%%js @out_error

if (window.IPython) {
    window.api_url = '';//http://localhost:8088'
} else {
    window.api_url = '';//http://localhost:8088'
}

// http://stackoverflow.com/questions/5916900/how-can-you-detect-the-version-of-a-browser
window.get_browser = function(){
    var ua=navigator.userAgent,tem,M=ua.match(/(opera|chrome|safari|firefox|msie|trident(?=\/))\/?\s*(\d+)/i) || []; 
    if(/trident/i.test(M[1])){
        tem=/\brv[ :]+(\d+)/g.exec(ua) || []; 
        return {name:'IE',version:(tem[1]||'')};
        }   
    if(M[1]==='Chrome'){
        tem=ua.match(/\bOPR\/(\d+)/)
        if(tem!=null) {return {name:'Opera', version:tem[1]};}
        }   
    M=M[2]? [M[1], M[2]]: [navigator.appName, navigator.appVersion, '-?'];
    if((tem=ua.match(/version\/(\d+)/i))!=null) {M.splice(1,1,tem[1]);}
    return M[0] + ' ' + M[1];
 }

function get_debug_info() {
    return {
        hash: window.hash,
        ipython: window.IPython,
        browser: get_browser()
    }
}

window.onerror = function(msg, url, line, col, error) {
    var message = "Error: " + msg + "\nurl: " + url + "\nline: " + line ;
    message += !col ? '' : '\ncolumn: ' + col;
    message += !error ? '' : '\nerror: ' + error;
    console.log(message);
    $.post(api_url + '/error', {
        page : window.location.href.split('#')[0],
        message : message,
        info : JSON.stringify(get_debug_info()),
    }, function() {
        alert('Произошла ошибка.\nИнформация отправлена администратору.\nПопробуйте перезагрузить страницу и повторить запрос.')
    }).fail(function() {
        alert('Не удается связаться с сервером.\n\
Проверьте соединение и перезагрузите эту страницу.\n\
Если соединение не прерывалось, то это проблема на стороне сервера.\n\
В этом случае напишите на pallada-92@ya.ru');
    });
    return true;
};

<IPython.core.display.Javascript object>

In [1019]:
%css /data/sets/static/visjs@4.16.1/dist/vis.min.css .visjs

### Data Manager

In [1020]:
%%jsx @out
window.obj_to_list = function(obj) {
    if (typeof obj != 'object') {
        return obj;
    }
    let keys = [];
    for (var key in obj) {
        keys.push(key);
    }
    keys.sort();
    let res = [];
    for (let i=0; i < keys.length; i++) {
        let key = keys[i];
        if (key == '' + i) {
            res.push(obj_to_list(obj[key]));
        } else {
            res.push([key, obj_to_list(obj[key])]);
        }
    }
    return res;
}

Starting react...


<IPython.core.display.Javascript object>

In [1021]:
%%jsx @out

window.tt = function(val, if_null, if_missing, ...path) {
    if (val === null) {
        return if_null;
    }
    let cur_obj = val;
    for (let i=0; i<path.length; i++) {
        let key = path[i];
        if (typeof(key) == 'function') {
            if (cur_obj == undefined || cur_obj == null) {
                return if_missing;
            } else {
                return key(cur_obj);
            }
        }
        if (key in cur_obj) {
            cur_obj = cur_obj[key];
        } else {
            return if_missing;
        }
    }
    if (cur_obj === undefined || cur_obj === null) {
        return if_missing
    }
    return cur_obj;
}

window.push_if_not_null = function(list, val) {
    if (val !== null) {
        list.push(val);
    }
}

window.iso_to_human = function(iso) {
    return iso.slice(0, 19).replace('T', ' ');
}

window.stamp_to_human = function(stamp) {
    let ms = (stamp - 0) * 1000;
    return iso_to_human(new Date(ms).toISOString());
}

<IPython.core.display.Javascript object>

In [1022]:
%%jsx @out

window.DataManager = class {
    constructor(chat_type, hash) {
        this.hash = hash
        this.chat_type = chat_type
        this.update_subscribers = []
        this._reset()
    }
    _reset() {
        this.cache = {}
        this.params = {
            word_id: null,
            user_id: null,
            graph: 'hide', //'communications', 'invitations', 'hide'
            temporal: 'hide', // 'week', 'dow', 'month', 'hour', 'hide'
            temporal_group: 'msg_count', //
            words: 'user', // 'all', 'user', 'month', 'week', 'dow', 'hour'
        }        
    }
    reset() {
        this._reset()
        this.on_update()
    }
    register_update(func) {
        this.update_subscribers.push(func)
        return this.update_subscribers.length - 1;
    }
    on_update() {
        render()
        for (var i=0; i<this.update_subscribers.length; i++) {
            let elem = this.update_subscribers[i];
            if (typeof elem == 'function') {
                elem();
            }
        }
    }
    off_update(pos) {
        this.update_subscribers[pos] = undefined;
    }
    param(name, val) {
        if (val === undefined) {
            return this.params[name];
        }
        if (R.equals(this.params[name], val)) return;
        this.params[name] = val;
        this.on_update();
    }
    gd(method, params) {
        let key = this.chat_type + this.hash + '/' + method + JSON.stringify(obj_to_list(params));
        if (key in this.cache) {
            return this.cache[key];
        }
        $.get(api_url + '/' + this.chat_type + this.hash + '/' + method, params).done((response) => {
            this.cache[key] = response;
            this.on_update();
        });
        return null;
    }
}

<IPython.core.display.Javascript object>

### Loadable components base

In [1023]:
%%jsx @out

window.LoadableComponent = class {
    constructor(element, dm, on_active_change, width, height) {
        this.width = width;
        this.height = height;
        this.element = element;
        this.dm = dm;
        this.dm_token = this.dm.register_update(this.data_update.bind(this));
        this.on_active_change = on_active_change;
        this.active = false;
        this.data = {};
    }
    // Return null, if not loaded
    get_data() {
        throw Error('Abstract');
    }
    start() {
        throw Error('Abstract');
    }
    stop() {
        throw Error('Abstract');
    }
    data_update() {
        let new_data = this.get_data();
        if (R.equals(new_data, this.data) && this.active) return;
        if (new_data == null) {
            if (this.active) {
                this.stop();
                this.active = false;
                this.on_active_change();
            }
            return;
        }
        if (this.active) {
            this.stop();
            console.log(this.data, new_data)
            this.data = new_data;
            this.start();
        } else {
            this.data = new_data;
            this.start();
            this.active = true;
            this.on_active_change();
        }
    }
    unmount() {
        this.dm.off_update(this.dm_token);
        if (this.active) {
            this.stop();
            this.active = false;
        }
    }
}

<IPython.core.display.Javascript object>

In [1024]:
%%jsx @out

window.ReactLoadableComponent = class extends React.Component {
    componentDidMount() {
        this.component = new this.props.componentClass(
            this.refs.element,
            window.dm,
            this.active_change.bind(this),
            this.props.width,
            this.props.height
        );
        this.active_change();
        this.component.data_update();
    }
    componentWillUnmount() {
        this.component.unmount();
    }
    active_change() {
        $(this.refs.loading).toggle(!this.component.active);
        $(this.refs.element).toggle(this.component.active);
    }
    render() {
        let height = this.props.height || 50;
        let style0 = {
            width: this.props.width,
            height: height,
            border: '1px solid lightgray',
            background: 'white',
        }
        let style1 = {
            textAlign: 'center',
            color: '#999',
            lineHeight: height
        }
        return <div style={style0}>
            <div style={style1} ref="loading">
                Загрузка...
            </div>
            <div ref="element"/>
        </div>
    }
}

<IPython.core.display.Javascript object>

### InvitationGraphComponent

In [1025]:
%%jsx @out

window.InvitationGraphComponent = class extends LoadableComponent {
    get_data() {
        let users = this.dm.gd('users')
        let invitations = this.dm.gd('invitations')
        if (users === null || invitations === null) return null;
        let nodes = R.values(R.mapObjIndexed((user, id) => ({
            id: id - 0,
            label: user.full_name,
            shape: 'circularImage',
            image: user.avatar || ''
        }), users));
        let edges = invitations.relations.map(({invited_by, user_id}) => ({
            from: invited_by - 0,
            to: user_id - 0,
            arrows: 'to',
        }))
        return {nodes, edges}
    }
    start() {
        let data1 = {
            nodes: new vis.DataSet(R.clone(this.data.nodes)),
            edges: new vis.DataSet(R.clone(this.data.edges))
        }
        let options = {
            width: this.width,
            height: this.height,
            clickToUse: true,
        }
        this.network = new vis.Network(this.element, data1, options)
    }
    stop() {
        this.network.destroy()
    }
}

<IPython.core.display.Javascript object>

### CommunicationsGraphComponent

In [1026]:
%%jsx @out

window.CommunicationsGraphComponent = class extends LoadableComponent {
    get_data() {
        var users = this.dm.gd('users', {});
        var graph = this.dm.gd('communications', {});
        if (graph === null || users === null) return null;
        let active_users = R.map((id) => id - 0, R.keys(R.filter(R.prop('msg_count'), users)));
        let nodes = R.values(R.mapObjIndexed(({full_name, msg_count, avatar}, id) => ({
            id: id - 0,
            label: full_name,
            shape: 'circularImage',
            size: 20,
            image: avatar || ''
        }), users));
        nodes = R.filter(({id}) => R.contains(id, active_users), nodes)
        var sliced_graph = graph.relations.slice(0, Math.round(1.6 * nodes.length));
        var edges = sliced_graph.map(({from_id, to_id, msg_count}) => {return {
            from: from_id - 0,
            to: to_id - 0,
            value: Math.log(msg_count) + 1
        }});
        edges = R.filter(({from, to}) => R.contains(from, active_users) && R.contains(to, active_users), edges)
        return {nodes, edges}
    }
    start() {
        let data1 = {
            nodes: new vis.DataSet(R.clone(this.data.nodes)),
            edges: new vis.DataSet(R.clone(this.data.edges))
        }
        let options = {
            width: this.width,
            height: this.height,
            clickToUse: true,
        }
        this.network = new vis.Network(this.element, data1, options)
    }
    stop() {
        this.network.destroy()
    }
}

<IPython.core.display.Javascript object>

### SortableTable

In [1027]:
%%css .datagrid @out
table {
    border-collapse: collapse;
    text-align: left;
    width: 100%;
    background: #fff;
}
table td, table th {
    padding: 6px 6px;
    color: #555!important;
}
table thead th {
    border-left: 1px solid #DDDDDD;
    border-bottom: 1px solid #DDDDDD;
    cursor: pointer;
}
table thead th:hover {
    background: #ddd;
}
table thead th:first-child {
    border: none;
    border-bottom: 1px solid #DDDDDD;
}
table tbody td {
    border-left: 1px solid #DDDDDD;
    border-bottom: 1px solid #DDDDDD;
}
table tbody td:first-child {
    border-left: none;
}
table tbody tr:last-child td {
    border-bottom: none;
}

In [1028]:
%%jsx @out
window.SortableTable = class extends React.Component {
    sort(column) {
        this.sort_by = column
        render()
    }
    render() {
        if (this.sort_by) {
            this.data = R.reverse(R.sortBy(R.path([this.sort_by, 1]), this.props.data))
            console.log(R.map(R.path([this.sort_by, 1]), this.data))
            console.log("After sort by " + this.sort_by, this.data)
        } else {
            this.data = this.props.data
        }
        this.columns = this.props.columns
        let t = R.transpose(this.props.columns)
        this.col_names = t[0] || []
        this.col_elems = t[1] || []
        let header = <thead><tr>
            {R.map(
                ([col_name, col_elem]) => {
                    let sel = col_name == this.sort_by
                    let style = {
                        fontWeight: sel ? 'bold' : 'normal'
                    }
                    let i_style = {
                        visibility: sel ? 'visible' : 'hidden'
                    }
                    let handler = this.sort.bind(this, col_name)
                    return <th style={style} onClick={handler} key={col_name}>
                        {col_elem} {' '}
                        <i className="fa fa-sort-desc" style={i_style}></i>
                    </th>
                },
                this.columns
            )}
        </tr></thead>
        let body = <tbody>
            {R.map(
                (row) => <tr key={row.id[1]}>
                    {R.map(
                        (col_name) => <td key={col_name}
                                style={{textAlign: typeof(row[col_name][1]) == 'number' ?
                                        'right' : 'left'}}
                            >{row[col_name][0]}</td>,
                        this.col_names
                    )}
                </tr>,
                this.data
            )}
        </tbody>
        return <div className="datagrid"><table>{header}{body}</table></div>
    }
}

<IPython.core.display.Javascript object>

### Selector

In [1029]:
%%jsx @out
window.Selector = function(props) {
    let selected_value = dm.param(props.paramName)
    let item_style = (sel) => ({
        display: 'inline-block',
        padding: '5px 10px',
        margin: '0px 3px',
        color: sel ? 'black' : null,
        fontSize: 15,
        cursor: sel ? 'normal' : 'pointer',
        //fontWeight: sel ? 400 : 300,
        borderRadius: 3,
        background: sel ? '#ffffaa' : 'white'
    })
    let handler = (value) => (() => 
        dm.param(props.paramName, value)
    )
    let get_item = ([value, text]) => {
        let sel = value == selected_value;
        return <div key={value} style={item_style(sel)}
        className={sel ? null : 'hover-cl'} onClick={handler(value)}>
            {text}
        </div>
    }
    return <div style={{textAlign: 'center', padding: 5}}>
        {props.items.map(get_item)}
    </div>
}

window.Title = function(props) {
    return <div style={{textAlign: 'center', padding: 20, fontSize: 18, color: '#999'}}>
        {props.text}
    </div>
}

window.Pad = function(props) {
    return <div style={{height: props.height}}></div>
}

<IPython.core.display.Javascript object>

### ChatInfo

In [1030]:
%%jsx @out
window.ChatInfo = function () {
    let chat_info = dm.gd('chat_info');
    let lines = [];
    let site = tt(chat_info, null, null, 'site')
    if (site == 'vk') {
        lines.push('Групповой чат ВК.')
    } else if (site == 'tg') {
        push_if_not_null(lines,
            tt(chat_info, null, 'Чат telegram.', 'chat_type', (ct) =>({
                'private': 'Приватный чат telegram.',
                'group': 'Групповой чат telegram.',
                'supergroup': 'Супергруповой чат telegram.',
                'channel': 'Канал telegram.',
            }[ct]))
        )
    }
    push_if_not_null(lines,
        tt(chat_info, null, null, 'uploaded_by', (ub) => 
           tt(dm.gd('users'), 'Загружено пользователем ...', null,
              ub, 'full_name', (fn) => 'Загружено пользователем ' + fn + '.'
            )
        )
    )
    push_if_not_null(lines,
        tt(chat_info, null, null, 'update_date', (ud) =>
           'Дата последнего обновления: ' + iso_to_human(ud) + '.'
        )
    )
    push_if_not_null(lines,
        tt(chat_info, null, null, 'uploaded_date', (ud) =>
           'Дата загрузки: ' + iso_to_human(ud) + '.'
        )
    )
    push_if_not_null(lines,
        tt(chat_info, 'Проанализировано ... сообщений',
        'Проанализированы сообщения', 'msg_count', (mc) =>
            'Проанализировано ' + mc + ' соощений'
        ) + 
        tt(chat_info, ' с ..', ' с ..', 'first_msg_stamp', (st) =>
            ' с ' + stamp_to_human(st)
        ) +
        tt(chat_info, ' по ..', ' по ..', 'last_msg_stamp', (st) =>
            ' по ' + stamp_to_human(st)
        ) + '.'
    )
    lines = lines.map((line) => <div>{line}</div>)
    return <div>
        <div style={{textAlign: 'left', fontSize: 18, color: 'black', padding: 15}}>
            {tt(chat_info, 'Загрузка...', 'Безымянный чат', 'title')}
        </div>
        <div style={{color: '#777', textAlign: 'left', fontSize: 13, padding: '0px 0px 15px 15px'}}>
            {lines}
        </div>
    </div>
}


<IPython.core.display.Javascript object>

### Graphs

In [1031]:
%%jsx @out
window.Graphs = function () {
    let items = [
        ['communications', 'Кто с кем общается'],
        ['invitations', 'Кто кого пригласил'],
        ['hide', 'Скрыть'],
    ]
    let graph = null;
    if (dm.param('graph') == 'invitations')  {
        graph = <ReactLoadableComponent key='invitations' componentClass={InvitationGraphComponent} width='100%' height='500px'/>
    } else if (dm.param('graph') == 'communications') {
        graph = <ReactLoadableComponent key='communications' componentClass={CommunicationsGraphComponent} width='100%' height='500px'/>        
    }
    return <div>
        <Selector paramName="graph" items={items}/>
        {graph}
    </div>
}

<IPython.core.display.Javascript object>

### Users

In [1032]:
%%jsx @out

window.float_to_human = function(float, frac_digits) {
    if (R.isNil(float) || isNaN(float)) {
        return '—'
    }
    if (float < 0) {
        return '-' + float_to_human(-float, frac_digits)
    }
    let int = Math.floor(float)
    let frac = float - int
    let t = '' + (1 + float  - int)
    let frac_str
    if (t.indexOf('.') == -1) {
        frac_str = ''
    } else {
        frac_str = t.split('.')[1]
    }
    frac_str += '00000000000'
    return int + '.' + frac_str.slice(0, frac_digits)
}

window.ratio_to_human = function (ratio) {
    if (R.isNil(ratio) || isNaN(ratio)) {
        return '—'
    } else {
        return float_to_human(ratio * 100, 2) + '%'
    }
}

window.Users = function () {
    let users = dm.gd('users')
    if (users === null) {
        return <div style={{color: '#999', padding: 20}}>
            Загрузка пользователей...
        </div>
    }
    let col_specs = [
         {
            name: 'profile_url',
            title: '',
            val: (val, user) => val && user.id,
            content: (val, user) => <a href={user.profile_url} target="_blank"><i className="fa fa-external-link"></i></a>
        }, {
            name: 'avatar',
            title: '',
            val: (val, user) => val && user.id,
            content: (val, user) => <img src={user.avatar} width={30} height={30}/>
        }, {
            name: 'full_name',
            title: '',
            val: (val, user) => val,
            content: (val, user) => val,
        }, {
            name: 'select',
            title: '',
            val: () => true,
            content: (val, user) => 
                <i className={
                    user.id == dm.param('user_id') ?
                    "fa fa-dot-circle-o fa-lg" : "fa fa-circle-o fa-lg"
                } style={{cursor: 'pointer'}} onClick={() => dm.param('user_id', user.id)}></i>
        }, {
            name: 'msg_count',
            title: 'сообщ.',
            val: (val) => val || null,
            content: (val, user) => R.isNil(val) ? '—' : val
        }, {
            name: 'adj_count',
            title: 'прил.',
            val: (val, user) => user.word_count && !R.isNil(val) ? val / user.word_count : null,
            content: (val, user) => ratio_to_human(val)
        }, {
            name: 'noun_count',
            title: 'сущ.',
            val: (val, user) => user.word_count && !R.isNil(val) ? val / user.word_count : null,
            content: (val, user) => ratio_to_human(val)
        }, {
            name: 'total_length',
            title: 'ср. длина',
            val: (val, user) => user.msg_count && !R.isNil(val) ? val / user.msg_count : null,
            content: (val, user) => float_to_human(val, 1)
        }, {
            name: 'punct_count',
            title: ';)',
            val: (val, user) => user.total_length && !R.isNil(val) ? val / user.total_length : null,
            content: (val, user) => ratio_to_human(val, 2)
        }, {
            name: 'id',
            title: null,
            val: (val) => val,
            content: (val) => null
        }
    ]
    let active_users = R.filter(R.prop('msg_count'), R.values(users))
    let inactive_users = R.reject(R.prop('msg_count'), R.values(users))
    let inactive_users_descr = <div style={{color: '#999', textAlign: 'left', padding: 10, fontSize: 13}}>
            Из данного отчета были исключены пользователи {' '}
            {R.intersperse(', ', inactive_users.map((user) => user.profile_url ?
                <a href={user.profile_url}>{user.full_name}</a> :
                <span>{user.full_name}</span>))}
            , так как за данный период они не оставили ни одного сообщения
    </div>
    if (inactive_users.length == 0) {
        inactive_users_descr = []
    }
    let data = R.map((user) =>
        R.fromPairs(R.map(({title, val, content, name}) => 
        [name, [
            content(val(user[name], user, users), user, users),
            val(user[name], user, users)
        ]], col_specs))
    , R.values(active_users))
    let columns = R.map(({title, name}) => [
        name, title
    ], col_specs)
    columns = R.reject(([name, title]) =>
        R.all(R.isNil, R.map(R.path([name, 1]), data))
    , columns)
    columns = R.reject(([name, title]) => R.isNil(title), columns)
    return <div>
        <Title text="Рейтинг пользователей"/>
        <SortableTable columns={columns} data={data}/>
        {inactive_users_descr}
    </div>
}

<IPython.core.display.Javascript object>

### Plots

In [1033]:
%%jsx @out 

window.two_digits = function(x) {
    if (x <= 9) {
        return '0' + ('' + x)
    }
    return '' + x;
}

window.add_date = function(date, days) {
    var t = new Date(date);
    t.setDate(t.getDate() + days);
    return t
}

window.date_range = function(by, min, max) {
    if (by == 'hour') {
        min = '00'
        max = '23'
    } else if (by == 'dow') {
        min = '0'
        max = '6'
    }
    let res = [min]
    let cur_date = min;
    while (cur_date < max) {
        if (by == 'hour') {
            cur_date = two_digits(cur_date - 0 + 1)
        } else if (by == 'dow') {
            cur_date = cur_date - 0 + 1 + ''
        } else if (by == 'month') {
            let [year, month] = cur_date.split('-').map((x) => x - 0)
            month += 1
            if (month > 12) {
                month = 1
                year += 1
            }
            cur_date = year + '-' + two_digits(month)
        } else if (by == 'week') {
            let t = add_date(cur_date, 7)
            cur_date = t.getFullYear() + '-' + two_digits(t.getMonth() + 1) + '-' + two_digits(t.getDate())
        }
        res.push(cur_date)
    }
    return res
}

window.TemporalPlotComponent = class extends LoadableComponent {
    get_data() {
        let by = this.dm.param('temporal')
        let user_id = this.dm.param('user_id')
        let word_id = this.dm.param('word_id')
        if (by == 'hide') {
            return null;
        }
        let users = this.dm.gd('users')
        let temporal = this.dm.gd('temporal', {by})
        let temporal_user = user_id && this.dm.gd('temporal_user', {by, user_id})
        let temporal_word = word_id && this.dm.gd('temporal_word', {by, word_id})
        if (temporal === null) return null;
        let active_attrs = R.mergeAll(R.values(temporal))
        let [min_date, max_date] = R.juxt([R.reduce(R.min, '9999-99-99'), R.reduce(R.max, '0000-00-00')])(R.keys(temporal))
        let dr = date_range(by, min_date, max_date)
        let items = []
        let groups = []
        let date_mapping = {
            week: (date) => add_date(date, 3),
            month: (month) => month + '-15',
            dow: (dow) => '2000-01-1' + (dow != '0' ? dow - 1 : 6) + ' 12:00',
            hour: (hour) => '2000-01-01 ' + hour + ':30'
        }[by]
        let shift_range = {
            week: [add_date(min_date, -1), add_date(max_date, 7)],
            month: [add_date(min_date, -0), add_date(max_date, 30)],
            dow: ['2000-01-10', '2000-01-17'],
            hour: ['2000-01-01 00:00', '2000-01-02 00:00'],
        }[by]
        if ('msg_count' in active_attrs) {
            dr.forEach((period) => items.push({
                x: date_mapping(period),
                y: R.path([period, 'msg_count'], temporal) || 0,
                group: 'msg_count'
            }))
            groups.push({
                id: 'msg_count',
                visible: this.dm.param('temporal_group') == 'msg_count'
            })
        }
        if ('active_users' in active_attrs) {
            dr.forEach((period) => items.push({
                x: date_mapping(period),
                y: R.path([period, 'active_users'], temporal) || 0,
                group: 'active_users'
            }))
            groups.push({
                id: 'active_users',
                visible: this.dm.param('temporal_group') == 'active_users'
            })
        }
        if ('total_punct' in active_attrs && 'total_chars' in active_attrs) {
            dr.forEach((period) => {
                let total_chars = R.path([period, 'total_chars'], temporal) || 0
                let total_punct = R.path([period, 'total_punct'], temporal) || 0                
                items.push({
                    x: date_mapping(period),
                    y: total_chars > 0 ? total_punct / total_chars : 0,
                    group: 'punct' 
                })
            })
            groups.push({
                id: 'punct',
                visible: this.dm.param('temporal_group') == 'punct'
            })
        }
        if (temporal_user !== null) {
            dr.forEach((period) => items.push({
                x: date_mapping(period),
                y: R.path([period, 'msg_count'], temporal_user) || 0,
                group: 'user_msg' 
            }))
            groups.push({
                id: 'user_msg',
                visible: this.dm.param('temporal_group') == 'user_msg'
            })
        }
        if (temporal_word !== null) {
            dr.forEach((period) => items.push({
                x: date_mapping(period),
                y: R.path([period, 'msg_count'], temporal_word) || 0,
                group: 'word_msg'
            }))
            groups.push({
                id: 'word_msg',
                visible: this.dm.param('temporal_group') == 'word_msg'
            })
        }
        return {items, groups, shift_range, by};
    }
    start() {
        let dataset = new vis.DataSet(R.clone(this.data.items))
        let groups = new vis.DataSet(R.clone(this.data.groups))
        let by = this.data.by
        let day = 24 * 60 * 60 * 1000
        let options = {
            width: this.width,
            height: this.height,
            clickToUse: true,
            min: this.data.shift_range[0],
            max: this.data.shift_range[1],
            start: this.data.shift_range[0],
            end: this.data.shift_range[1],
            zoomMin: {
                week: 7 * day,
                month: 30 * day,
                dow: day,
                hour: day / 24
            }[by],
            showMajorLabels: {
                week: true,
                month: true,
                dow: false,
                hour: false
            }[by],
            maxMinorChars: by == 'week' ? 3 : 3,
            format: {
                  minorLabels: {
                    hour:  'HH',
                    weekday:  by == 'dow' ? 'ddd' : 'D',
                    day: {
                        hour: 'D',
                        dow: 'ddd',
                        week: 'DD',
                    }[by],
                    month:  'MMM',
                    year: 'YYYY'
                  }
            }
        }
        this.plot = new vis.Graph2d(this.element, dataset, groups, options)
    }
    stop() {
        this.plot.destroy()
    }
}

<IPython.core.display.Javascript object>

In [1034]:
%%jsx @out
window.Plots = function () {
    let periods = [
        ['hour', 'По времени суток'],
        ['dow', 'По дням недели'],
        ['week', 'По неделям'],
        ['month', 'По месяцам'],
        ['hide', 'Скрыть'],
    ]
    let groups = [
        ['msg_count', 'Всех сообщ.'],
        ['active_users', 'Активных пользователей'],
        ['punct', ';)']
    ]
    if (dm.param('user_id') && dm.gd('users') !== null) {
        let full_name = dm.gd('users')[dm.param('user_id')].full_name
        groups.push(['user_msg', 'Сообщ. ' + full_name])
    }
    if (dm.param('word_id')) {
        groups.push(['word_msg', 'Сообщ. "' + dm.param('word') + '"'])
    }
    let component = <ReactLoadableComponent componentClass={TemporalPlotComponent} width='100%' height='500px'/>
    return <div className="visjs" style={{position: 'relative'}}>
        <Selector paramName="temporal" items={periods}/>
        <Selector paramName="temporal_group" items={groups}/>
        {dm.param('temporal') == 'hide' ? null : component}
    </div>
}

<IPython.core.display.Javascript object>

### Words

In [1035]:
%%jsx @out

window.WordsList = function(props) {
    let make_handler = (word_id, word) => () => {
        dm.param('word_id', word_id)
        dm.param('word', word)
    }
    let style = (sel) => ({
        color: sel ? 'black' : 'blue',
        fontWeight: sel ? 'bold' : 'normal',
        cursor: sel ? 'normal' : 'pointer',
    })
    return <div style={{width: props.width, display: 'inline-block',
            textAlign: 'left', padding: '10px 5px', verticalAlign: 'top'}}>
        <div style={{fontSize: 18, color: '#999'}}>{props.title}</div>
        {R.intersperse(', ', props.words.map(({word, word_id, msg_count}) =>
            <span style={style(word_id == dm.param('word_id'))}
                  onClick={make_handler(word_id, word)}>
                {word}<sup>{msg_count}</sup>
            </span>,
            props.words
        ))}
    </div>
}

window.Words = function () {
    let groups = [
        ['all', 'Все слова'],
        ['dow', 'По дням недели'],
        ['month', 'По месяцам'],
        ['user', 'По пользователям']
    ]
    let loading = <div style={{color: '#999', padding: 20}}>
        Загрузка слов...
    </div>
    let blocks = []
    let by = dm.param('words')
    if (by == 'all') {
        let top_words = dm.gd('top_words')
        if (top_words === null) return loading
        blocks = [<WordsList title="" width="100%" words={top_words}/>]
    } else if (by == 'dow' || by == 'month') {
        let top_words = dm.gd('temporal_top_words', {by})
        if (top_words === null) return loading
        let months = ['Январь', 'Февраль', 'Март', 'Апрель',
                      'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
                      'Октябрь', 'Ноябрь', 'Декабрь']
        let days_of_week = ['Воскресенье', 'Понедельник', 'Вторник',
                            'Среда', 'Четверг', 'Пятница', 'Суббота']
        let display_key = {
            dow: (key) => days_of_week[key - 0],
            month: (key) => months[key.split('-')[1] - 1] + ' ' + key.split('-')[0]
        }[by]
        blocks = R.map(([key, obj]) => 
            <WordsList title={display_key(key)} width={250} words={obj}/>
        , R.sortBy(R.prop(0), R.toPairs(top_words)))
    } else if (by == 'user') {
        let top_words = dm.gd('users_top_words')
        let users = dm.gd('users')
        if (top_words === null || users === null) {
            return loading
        }
        blocks = R.map(([full_name, obj]) => 
            <WordsList title={full_name} width={240} words={obj}/>
        , R.sortBy(R.prop(0), R.map(([user_id, obj]) =>
            [users[user_id].full_name, obj]
        , R.toPairs(top_words))))
    }
    return <div>
        <Selector paramName="words" items={groups}/>
        {blocks}
    </div>
}

<IPython.core.display.Javascript object>

### Application

In [1036]:
%%css .application @out
.hover-cl {
    color: #999
}
.hover-cl:hover {
    color: black
}
svg:not(:root) {
    overflow: visible;
}

In [1037]:
%%jsx @out
window.Application = function () {
    return <div style={{width: '100%', maxWidth: 800, display: 'inline-block', fontFamily: 'Roboto, Helvetica', background: 'white'}}>
        <ChatInfo/>
        <Pad height={60}/>
        <Graphs/>
        <Pad height={60}/>
        <Users/>
        <Pad height={60}/>
        <Plots/>
        <Pad height={60}/>
        <Words/>
        <Pad height={60}/>
    </div>
}

<IPython.core.display.Javascript object>

In [1038]:
%%html @body
<div id="root" class="application" style="text-align: center"></div>

### Render

In [1039]:
%%jsx @out
window.render = function() {
    let chat_info = dm.gd('chat_info')
    if (chat_info !== null && chat_info.title) {
        document.title = chat_info.title
    }
    ReactDOM.render(
        <Application/>,
        document.getElementById('root')
    );
}

<IPython.core.display.Javascript object>

In [1040]:
%%js @out_run
if (!window.IPython) {
    $(function() {
        let chat_code = window.location.href.split('/').pop()
        let site = chat_code.slice(0, 2)
        let chat_id = chat_code.slice(2)
        window.dm = new DataManager(site, chat_id);
        render();
    })
}

<IPython.core.display.Javascript object>